<a href="https://colab.research.google.com/github/atinrov/PAIthology/blob/main/Entrenamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone "https://github.com/google/automl.git"
!pip install -r /kaggle/working/automl/efficientdet/requirements.txt


import sys
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2
import pandas as pd
import ast
import glob
import sys
import tensorflow.compat.v1 as tf


!curl -L "https://app.roboflow.com/ds/ov346wjjfH?key=X9e3bSz1F5" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

MODEL = 'efficientdet-d0'  #@param

def download(m):
  if m not in os.listdir():
    !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientdet/coco/{m}.tar.gz
    !tar zxf {m}.tar.gz
  ckpt_path = os.path.join(os.getcwd(), m)
  return ckpt_path

# Download checkpoint.
ckpt_path = download(MODEL)
print('Use model in {}'.format(ckpt_path))


# Train efficientdet from scratch with backbone checkpoint.
backbone_name = {
    'efficientdet-d0': 'efficientnet-b0',
    'efficientdet-d1': 'efficientnet-b1',
    'efficientdet-d2': 'efficientnet-b2',
    'efficientdet-d3': 'efficientnet-b3',
    'efficientdet-d4': 'efficientnet-b4',
    'efficientdet-d5': 'efficientnet-b5',
    'efficientdet-d6': 'efficientnet-b6',
    'efficientdet-d7': 'efficientnet-b6',
}[MODEL]


# generating train tfrecord is large, so we skip the execution here.
import os
if backbone_name not in os.listdir():
  !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/ckptsaug/{backbone_name}.tar.gz
  !tar xf {backbone_name}.tar.gz

!mkdir /kaggle/working/automl/efficientdet/model_dir


#Cambiar patha /kaggle/working/train/' poner la carpegta de los tf records

file_pattern = 'mitotic-cells.tfrecord'  # @param
images_per_epoch = 57 * len(tf.io.gfile.glob('/kaggle/working/train/' + file_pattern))
#¡images_per_epoch = images_per_epoch // 8 * 8  # round to 64.
print('images_per_epoch = {}'.format(images_per_epoch))


#Añasir el path dek archivo del tfrecord

!python main.py --mode=train_and_eval \
    --train_file_pattern=/kaggle/working/train/mitotic-cells.tfrecord \
    --val_file_pattern=/kaggle/working/valid/mitotic-cells.tfrecord \
    --model_name={MODEL} \
    --model_dir=/tmp/model_dir/{MODEL}-scratch  \
    --backbone_ckpt={backbone_name} \
    --train_batch_size=4 \
    --eval_batch_size=4 --eval_samples={images_per_epoch}  \
    --num_examples_per_epoch={images_per_epoch}  --num_epochs=1  \
    --hparams="num_classes=20,moving_average_decay=0,mixed_precision=true"